# 05_deploy.ipynb

**Batch Inference Demo via Flask Endpoint**

This notebook loads a small sample of the test set from S3, converts it to JSON, and sends it to your locally‐running Flask‐based `/predict` endpoint. It then prints out the returned array of predicted annual electricity usages.

In [2]:
import requests, pandas as pd
from sagemaker import Session
import numpy as np

sess   = Session()
bucket = sess.default_bucket()
prefix = "recs"

X_test = pd.read_parquet(f"s3://{bucket}/{prefix}/test/X_test.parquet")

# pick a small sample
df = X_test.head(10)

# convert any NaNs to Python None (which serializes to JSON null)
df = df.where(pd.notnull(df), None)

records = df.to_dict(orient="records")

resp = requests.post(
    "http://127.0.0.1:8080/predict",
    json=records,
    headers={"Content-Type": "application/json"},
)
resp.raise_for_status()
print(resp.json())

{'predictions': [4979.5048828125, 19953.9921875, 5646.560546875, 15137.8427734375, 23397.421875, 7680.72900390625, 10778.5087890625, 6094.0478515625, 3698.609130859375, 13222.3369140625]}
